In [1]:
%matplotlib inline
import os
import glob
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [2]:
url = 'https://worksheets.codalab.org/rest/bundles/0x9d8c9e97326746459b4eb6564324d444/contents/blob/reddit_results_reformatted.csv'
df = pd.read_csv(url, header=0, index_col=None)
print(df.shape)
df.head(n=3)

(480, 135)


,Input.TYPE0,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,AutoApprovalDelayInSeconds,Expiration,NumberOfSimilarHITs,LifetimeInSeconds,AssignmentId,WorkerId,AssignmentStatus,AcceptTime,SubmitTime,AutoApprovalTime,ApprovalTime,RejectionTime,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.NAME,Input.CTX0,Input.OUT0,Input.LOGP0,Input.CTX1,Input.OUT1,Input.LOGP1,Input.CTX2,Input.OUT2,Input.LOGP2,Input.CTX3,Input.OUT3,Input.LOGP3,Input.CTX4,Input.OUT4,Input.LOGP4,Input.CTX5,Input.OUT5,Input.LOGP5,Input.CTX6,Input.OUT6,Input.LOGP6,Input.CTX7,Input.OUT7,Input.LOGP7,Input.CTX8,Input.OUT8,Input.LOGP8,Input.CTX9,Input.OUT9,Input.LOGP9,Input.CTX10,Input.OUT10,Input.LOGP10,Input.CTX11,Input.OUT11,Input.LOGP11,Input.CTX12,Input.OUT12,Input.LOGP12,Input.CTX13,Input.OUT13,Input.LOGP13,Input.CTX14,Input.OUT14,Input.LOGP14,Input.CTX15,Input.OUT15,Input.LOGP15,Input.CTX16,Input.OUT16,Input.LOGP16,Input.CTX17,Input.OUT17,Input.LOGP17,Input.CTX18,Input.OUT18,Input.LOGP18,Input.CTX19,Input.OUT19,Input.LOGP19,Input.CTX20,Input.OUT20,Input.LOGP20,Input.CTX21,Input.OUT21,Input.LOGP21,Input.CTX22,Input.OUT22,Input.LOGP22,Input.CTX23,Input.OUT23,Input.LOGP23,Input.CTX24,Input.OUT24,Input.LOGP24,Input.CTX25,Input.OUT25,Input.LOGP25,Answer.0,Answer.1,Answer.10,Answer.11,Answer.12,Answer.13,Answer.14,Answer.15,Answer.16,Answer.17,Answer.18,Answer.19,Answer.2,Answer.20,Answer.21,Answer.22,Answer.23,Answer.24,Answer.25,Answer.3,Answer.4,Answer.5,Answer.6,Answer.7,Answer.8,Answer.9,Approve,Reject
0,HUMAN_REDDIT1.0,34D9ZRXCYRO1DRH8S8VPQ45YIPJSAS,3JD5BATOXQP1AP897HR0YZ4RGZM3VM,Estimate Sentence Frequency for Reddit,"Given a Reddit comment, estimate how often you...","survey, NLP, probability",$1.00,Fri Nov 16 23:05:03 PST 2018,20,BatchId:3439994;OriginalHitTemplateId:920937332;,3600,259200,Fri Nov 23 23:05:03 PST 2018,NaN,NaN,30LSNF239VPQ0UH10KDX44BZ83TI2A,A1ET2J1PIP0RGO,Submitted,Fri Nov 16 23:06:29 PST 2018,Fri Nov 16 23:14:59 PST 2018,Mon Nov 19 23:14:59 PST 2018,NaN,NaN,NaN,510,100% (64/64),100% (50/50),100% (44/44),GOLD_1.0.txt,*golf clap*,*slow fap*,-3.281735,andy?,andy the android,-17.912071,"aww, wrong selling point for me. nice try, tho...",we also have french dudes?,-40.171869,being a nice guy = christian thing ? hahaha,christians seem to think so...,-17.245636,but... but... *why???*,some place to go and just reflect.,-44.366392,can an elevator pick up the hammer?,is the elevator worthy?,-14.340388,"depends if the water is flowing. also, it may ...",he drove there. the white car is his. ^^^^/s,-53.332370,"dudes be careful, i think this guy might be a ...",who graduated top of his class.,-22.764982,i bet you guys party hard. and by that i mean ...,"hey you kids, get off my lan!",-23.085698,i was confused by that too! i think he left a ...,seemed like michael made out with the girl wit...,-171.831727,if it gets bad,'no new friends',-22.951707,if only the playoff system was this year...,everyone would hate stanford.,-26.221197,loved. you loved this little vampire dude.,='( r.i.p,-27.919784,must be a non-science major.,or more intelligent than you,-24.544617,"please please please, no one pour milk into ho...",too late,-12.598184,same here. that is the advantage of using abso...,wow. i hope this is the next big trend in web ...,-41.953097,sounds like her head game is sloppy,the best kind of head game,-15.123610,"technically, nor do the japanese",australian water,-20.899671,thank you! what language is this?,mexican scottish,-14.760982,that was actually better than expected,that was about what i expected,-18.275426,three times in the same set,wow...,-11.374801,too sweet!,too furious,-10.496938,"wait, am i supposed to wear a helmet in the car?",not unless you need it,-17.298362,what a depressing game,naw,-11.614001,who would win in a fight- mario or iron man?,my uncle works at nintendo and he actually bro...,-159.284697,

In [3]:
responses = ["Invalid", "Rare", "Specific", "Average", "Typical", "Very Typical"]
response_dict = dict(zip(responses, range(len(responses))))

In [4]:
worker_counter = Counter(df['WorkerId'].values)
print(len(worker_counter))
worker_dict = dict(zip(sorted(worker_counter), range(len(worker_counter))))

123


In [5]:
ctx_counter = Counter()
for i in range(25):
    ctx_counter.update(df['Input.CTX' + str(i)].values)
print(len(ctx_counter))
ctx_dict = dict(zip(sorted(ctx_counter), range(len(ctx_counter))))

99


In [6]:
model_counter = Counter()
model_counter.update(df['Input.NAME'].values)
print(model_counter)
model_dict = dict(zip(sorted(model_counter), range(len(model_counter))))

Counter({'GOLD_1.0.txt': 240, 'LOGP_1.0.txt': 80, 'LOGP_0.7.txt': 80, 'LOGP_0.5.txt': 80})


In [7]:
combo_counter = defaultdict(Counter)
for i in df.index:
    for j in range(25):
        combo_counter[df.loc[i, 'Input.CTX' + str(j)]].update([(df.loc[i, 'Input.NAME'], df.loc[i, 'Input.OUT' + str(j)])])
print(len(combo_counter))

99


In [8]:
for key, counter in combo_counter.items():
    print(key)
    for pair, count in counter.items():
        print(pair, count)
    print()

*golf clap*
('GOLD_1.0.txt', '*slow fap*') 60
('LOGP_1.0.txt', 'UNKNOWN  UNKNOWN  wrap! UNKNOWN') 20
('LOGP_0.7.txt', '*a cop suddenly gets a medal of honor. *') 20
('LOGP_0.5.txt', '*slow fap*') 20

andy?
('GOLD_1.0.txt', 'andy the android') 60
('LOGP_1.0.txt', "'carrot for account of relevant username'") 20
('LOGP_0.7.txt', 'please someone get me some flair war') 20
('LOGP_0.5.txt', 'andy?') 20

aww, wrong selling point for me. nice try, though!
('GOLD_1.0.txt', 'we also have french dudes?') 60
('LOGP_1.0.txt', 'i ...sagan...kane..') 20
('LOGP_0.7.txt', 'and kinda sad to think about it. :p') 20
('LOGP_0.5.txt', 'i actually thought it was funny.') 20

being a nice guy = christian thing ? hahaha
('GOLD_1.0.txt', 'christians seem to think so...') 60
('LOGP_1.0.txt', 'in saudi arabia? yeah no') 20
('LOGP_0.7.txt', 'i think he was joking. just saying. :)') 20
('LOGP_0.5.txt', "'nice guy' = 'nice guy'") 20

but... but... *why???*
('GOLD_1.0.txt', 'some place to go and just reflect.') 60
('

In [9]:
combo_counter = defaultdict(Counter)
rows = []
responses_by_sent_by_model = {model: defaultdict(list) for model in model_dict}

for i in df.index:
    worker = df.loc[i, 'WorkerId']
    worker_id = 'W' + str(worker_dict[worker])
    model = df.loc[i, 'Input.NAME']
    for j in range(25):
        ctx = df.loc[i, 'Input.CTX' + str(j)]
        item_id = 'I' + str(ctx_dict[ctx])
        output = df.loc[i, 'Input.OUT' + str(j)]
        response = df.loc[i, 'Answer.' + str(j)]
        response_val = float(response_dict[response])
        rows.append([worker, worker_id, model, ctx, item_id, output, response, response_val])
        responses_by_sent_by_model[model][output].append(response)

df_out = pd.DataFrame(rows, columns=['worker', 'worker_id', 'model', 'input', 'input_id', 'output', 'response', 'response_val'])
df_out.head()

,worker,worker_id,model,input,input_id,output,response,response_val
0,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,*golf clap*,I3,*slow fap*,Rare,1.0
1,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,andy?,I9,andy the android,Rare,1.0
2,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,"aww, wrong selling point for me. nice try, tho...",I14,we also have french dudes?,Rare,1.0
3,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,being a nice guy = christian thing ? hahaha,I15,christians seem to think so...,Rare,1.0
4,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,but... but... *why???*,I17,some place to go and just reflect.,Specific,2.0


In [10]:
print(set(df_out['model'].values))

{'GOLD_1.0.txt', 'LOGP_0.5.txt', 'LOGP_0.7.txt', 'LOGP_1.0.txt'}


In [12]:
model_code_map = {'GOLD_1.0.txt': 1, 'LOGP_0.5.txt': 2, 'LOGP_0.7.txt': 3, 'LOGP_1.0.txt': 4}
model_codes = [model_code_map[m] for m in df_out['model'].values]
df_out['model_code'] = model_codes
df_out.head()

,worker,worker_id,model,input,input_id,output,response,response_val,model_code
0,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,*golf clap*,I3,*slow fap*,Rare,1.0,1
1,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,andy?,I9,andy the android,Rare,1.0,1
2,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,"aww, wrong selling point for me. nice try, tho...",I14,we also have french dudes?,Rare,1.0,1
3,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,being a nice guy = christian thing ? hahaha,I15,christians seem to think so...,Rare,1.0,1
4,A1ET2J1PIP0RGO,W13,GOLD_1.0.txt,but... but... *why???*,I17,some place to go and just reflect.,Specific,2.0,1


In [13]:
outdir = os.path.join('..', 'data', 'annotated_datasets')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df_out.to_csv(os.path.join(outdir, 'huse_reddit.csv'))